*Copyright (c) Cornac Authors. All rights reserved.*

*Licensed under the Apache 2.0 License.*

# Model Ensembling

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/PreferredAI/cornac/blob/master/tutorials/model_ensembling.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/PreferredAI/cornac/blob/master/tutorials/model_ensembling.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

This notebook provides an example of how to ensemble multiple recommendation models in Cornac.

Ensemble models is a technique that combines the predictions of multiple models to produce a single prediction. The idea is that by combining the predictions of multiple models, we can improve the overall performance of the recommendation system.

We will use the MovieLens 100K dataset and ensemble 2 models.

** Note: ** Part of this notebook (in Section 4) uses the `scikit-learn` package. 

## 1. Setup

### 1.1 Install required dependencies

In [1]:
! pip install seaborn scikit-learn

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 3.7 MB/s eta 0:00:00
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.7/163.7 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 49.6 MB/s eta 0:00:00m eta 0:00:010:00:01
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 138.1 MB/s eta 0:00:001m205.5 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 94.4 MB/s eta 0:00:000m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
from IPython.display import display
import numpy as np
import pandas as pd

from cornac.datasets import movielens
from cornac.models import BPR, WMF
from cornac.eval_methods import RatioSplit
from cornac.metrics import Precision, Recall
from cornac.utils import cache
from cornac import Experiment

from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor

/home/darrylong/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Prepare Experiment

### 2.1 Loading Dataset

First, we load the MovieLens 100K dataset.

In [2]:
data = movielens.load_feedback(variant="100K") # Load MovieLens Dataset

rs = RatioSplit(data, test_size=0.2, seed=42, verbose=True) # Split to train-test set to 80-20
train_set, test_set = rs.train_set, rs.test_set

rating_threshold = 1.0
exclude_unknowns = True
---
Training data:
Number of users = 943
Number of items = 1651
Number of ratings = 80000
Max rating = 5.0
Min rating = 1.0
Global mean = 3.5
---
Test data:
Number of users = 943
Number of items = 1651
Number of ratings = 19964
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 943
Total items = 1651


### 2.2 Training BPR and WMF models

We will train two models: 

1. BPR (Bayesian Personalized Ranking)
2. WMF (Weighted Matrix Factorization)

In [4]:
bpr_model = BPR(k=10, max_iter=100, learning_rate=0.01, lambda_reg=0.001) # Initialize BPR model
wmf_model = WMF(k=10, max_iter=300, a=1.0, b=0.1, learning_rate=0.001, lambda_u=0.01, lambda_v=0.01,) # Initialize WMF model

models = [bpr_model, wmf_model]
metrics = [Precision(k=50), Recall(k=50)] # Set metrics for experiment

experiment = Experiment(rs, models, metrics, user_based=True).run() # Run Experiment to compare BPR model to WMF model individually


[BPR] Training started!

[BPR] Evaluation started!


Ranking: 100%|██████████| 940/940 [00:00<00:00, 1536.79it/s]



[WMF] Training started!


I0000 00:00:1728143120.745328   49677 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
100%|██████████| 300/300 [00:04<00:00, 61.13it/s, loss=172]


Learning completed!

[WMF] Evaluation started!


Ranking: 100%|██████████| 940/940 [00:00<00:00, 1740.02it/s]


TEST:
...
    | Precision@50 | Recall@50 | Train (s) | Test (s)
--- + ------------ + --------- + --------- + --------
BPR |       0.1823 |    0.5087 |    0.2548 |   0.6144
WMF |       0.1744 |    0.5122 |    6.3755 |   0.5425



Comparing Precision and Recall, both BPR and WMF are providing comparable results.

Let's move on to try to interpret these results by using the genres of movies that were recommended to us.

Generally, we could assume that if an individual likes a particular film genre like 'Romance', the recommender system should provide more of such 'Romance' films.

### 2.3 Interpreting Results

##### Creating a Movie Genre Dataframe

In [5]:
# Creating a dataframe of movies with its corresponding genres

# Download some information of MovieLens 100K dataset
item_df = pd.read_csv(
  cache("http://files.grouplens.org/datasets/movielens/ml-100k/u.item"), 
  sep="|", encoding="ISO-8859-1",
  names=["ItemID", "Title", "Release Date", "Video Release Date", "IMDb URL", 
         "unknown", "Action", "Adventure", "Animation", "Children's", "Comedy", 
         "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", 
         "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"]
).set_index("ItemID").drop(columns=["Video Release Date", "IMDb URL", "unknown"])

item_idx2id = train_set.item_ids # create a item index to film ID mapping
user_idx2id = train_set.user_ids

# Let's take a look at an example of this dataframe
display(item_df.head(3))


Data from http://files.grouplens.org/datasets/movielens/ml-100k/u.item
will be cached into /home/darrylong/.cornac/u.item


246kB [00:01, 127kB/s]                             

File cached!


,Title,Release Date,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
ItemID,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),01-Jan-1995,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),01-Jan-1995,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),01-Jan-1995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


The `item_df` dataframe consists of all movie items with its corresponding genre attributes.

Further down below, we are going to filter this table with the recommendations that we get from the recommender system models we created to get a better sense of its performance.

##### Creating Training Data Dataframe

To get a sense of what data has been inserted into our model for training, let's count the genres of the training data used to train the model.

But first, let's create a `training_data_df` dataframe with all training data.

The training data consists of 80000 triplets of **User Index**, **Item Index** and **Rating** rows as seen in the dataset summary in Section 2.1.

In [6]:
# Let's view a sample of the training data dataframe
print("Sample row of record:")
print("(user_index, item_index, rating):", list(zip(*train_set.uir_tuple))[0])

# Create a training data dataframe
training_data_df = pd.DataFrame(zip(*train_set.uir_tuple)) # adding all training data into dataframe
training_data_df.columns = ['user_idx', 'item_idx', 'rating'] # adding column names to the data

# Add new column, 'item_id', for further filtering in later sections
training_data_df['item_id'] = training_data_df.apply(lambda row: item_idx2id[int(row['item_idx'])], axis=1) # converted from the item index field

Sample row of record:
(user_index, item_index, rating): (0, 0, 4.0)


##### Filtering Training Data

Let's filter based on a particular user to learn more about the user.

We set ``UIDX`` to user index **3**, and ``TOPK`` to **50**, to get the top 50 recommendations in each model for comparison.

In [7]:
UIDX = 3
TOPK = 50

In [8]:
# Filter training data (rating = 5.0 and user index = UIDX)
filter_df = training_data_df[(training_data_df['rating'] == 5.0) & (training_data_df['user_idx'] == UIDX)]
filter_df = item_df.loc[[int(item_id) for item_id in filter_df["item_id"]]] # get genres of movie items

print("Number of movies:", len(filter_df))

# Group by Movie Genre and Sum by genres
filter_df = filter_df.select_dtypes(np.number).sum() 
filter_df = filter_df.to_frame("Sum") # Let's call that column 'Sum'

# Add a new column '%' for the percentage of individual genre sum compared to total sum
filter_df["%"] = filter_df["Sum"] / filter_df["Sum"].sum() * 100
filter_df["%"] = filter_df["%"].round(1)

# Let's see the training data genres, sums and percentages
print("Movies rated 5.0 by user index 3 in training data")
display(filter_df.sort_values("Sum", ascending=False))

Number of movies: 138
Movies rated 5.0 by user index 3 in training data


,Sum,%
Drama,71,24.9
Comedy,39,13.7
Romance,32,11.2
Action,30,10.5
Thriller,29,10.2
Adventure,19,6.7
War,15,5.3
Crime,12,4.2
Sci-Fi,9,3.2
Mystery,8,2.8


As shown above in the training data, the top genres for user index 3 with movies rated 5.0 include 'Drama', 'Comedy', 'Romance', 'Action' and 'Thriller'.

Let's now compare them to the recommendations of the BPR and WMF models respectively.

##### Interpreting Recommendations of BPR, WMF Models

In [9]:
# Get the Top 5 Genres in filtered training data for user index 3
top_genres = filter_df.sort_values("Sum", ascending=False).head(5).index.tolist()
print("\nTop 5 Genres in training data:", top_genres)

# Get top K recommendations for BPR and put them into the genre dataframe
bpr_recommendations, bpr_scores = bpr_model.rank(UIDX, k=TOPK) # rank recommendations by score, limit to top K
print(len(bpr_recommendations))
bpr_topk = [item_idx2id[iidx] for iidx in bpr_recommendations[:TOPK]] # convert item indexes into item ids
bpr_df = item_df.loc[[int(iid) for iid in bpr_topk]] # filter the movie genre dataframe by item ids

# Let's view the top recommendations for BPR by top genres
display("BPR: Top recommendations", bpr_df[["Title"] + top_genres].head(10))

# Now, let's do likewise for WMF - get top K recommendations and put them into the genre dataframe
wmf_recommendations, wmf_scores = wmf_model.rank(UIDX, k=TOPK) # rank recommendations by score
wmf_topk = [item_idx2id[iidx] for iidx in wmf_recommendations[:TOPK]] # convert item indexes into item ids
wmf_df = item_df.loc[[int(iid) for iid in wmf_topk]] # filter the movie genre dataframe by item ids

# View the top recommendations for WMF
display("WMF: Top recommendations", wmf_df[["Title"] + top_genres].head(10))


Top 5 Genres in training data: ['Drama', 'Comedy', 'Romance', 'Action', 'Thriller']
1651


'BPR: Top recommendations'

,Title,Drama,Comedy,Romance,Action,Thriller
ItemID,,,,,,
402,Ghost (1990),0,1,1,0,1
70,Four Weddings and a Funeral (1994),0,1,1,0,0
237,Jerry Maguire (1996),1,0,1,0,0
216,When Harry Met Sally... (1989),0,1,1,0,0
66,While You Were Sleeping (1995),0,1,1,0,0
283,Emma (1996),1,0,1,0,0
275,Sense and Sensibility (1995),1,0,1,0,0
173,"Princess Bride, The (1987)",0,1,1,1,0
255,My Best Friend's Wedding (1997),0,1,1,0,0


'WMF: Top recommendations'

,Title,Drama,Comedy,Romance,Action,Thriller
ItemID,,,,,,
313,Titanic (1997),1,0,1,1,0
318,Schindler's List (1993),1,0,0,0,0
272,Good Will Hunting (1997),1,0,0,0,0
50,Star Wars (1977),0,0,1,1,0
258,Contact (1997),1,0,0,0,0
268,Chasing Amy (1997),1,0,1,0,0
64,"Shawshank Redemption, The (1994)",1,0,0,0,0
181,Return of the Jedi (1983),0,0,1,1,0
100,Fargo (1996),1,0,0,0,1


Now that we have seen the top recommendations of the BPR and WMF models, let's do a comparison by taking a look at the genre distribution.

##### Comparing Models by Genre Distribution

In [10]:
# Let's introduce `combined_df` for comparison.
# This dataframe will be used to compare models by summing up genres from recommendations of different models
combined_df = pd.DataFrame({
    "Train Data %": filter_df["%"],
    "BPR Sum": bpr_df.select_dtypes(np.number).sum(), # group by genres, then get sum of each genre
    "WMF Sum": wmf_df.select_dtypes(np.number).sum() # likewise for WMF
})

# Get percentages of movie genre sums
combined_df['BPR %'] = combined_df['BPR Sum'] / combined_df['BPR Sum'].sum() * 100 
combined_df["WMF %"] = combined_df["WMF Sum"] / combined_df["WMF Sum"].sum() * 100

combined_df = combined_df.round(1) # round all 
combined_df = combined_df.sort_values("Train Data %", ascending=False)

# Let's take a look at the genre distribution by percentages
display("Train Data to Recommended % Distribution", combined_df[['Train Data %', 'BPR %', 'WMF %']])

'Train Data to Recommended % Distribution'

,Train Data %,BPR %,WMF %
Drama,24.9,12.4,31.0
Comedy,13.7,25.7,5.2
Romance,11.2,25.7,14.7
Action,10.5,10.6,8.6
Thriller,10.2,5.3,7.8
Adventure,6.7,6.2,4.3
War,5.3,4.4,8.6
Crime,4.2,0.9,3.4
Sci-Fi,3.2,5.3,4.3
Mystery,2.8,0.0,1.7


Now that we have seen the distribution of individual models, we are curious about what kind of distribution we will get from ensembling these models.

Let's see what happens when we ensemble these two models. 

## 2. Simple model ensembling by Borda Count

We will ensemble the two models using the Borda Count method. The Borda Count method is a simple voting method that ranks the items based on the sum of their ranks from each model.

Assuming that we have a list of 5 items, the Borda Count method works as follows:

1. For each model, rank the items from 1 to 5 based on the predicted scores.
2. Sum the ranks of each item across all models.
3. Sort the items based on the sum of their ranks.
4. The top-ranked item is the final recommendation.
5. Repeat the process for the next user.

Given the below example for a random user 123:

| Rank | Model 1 | Model 2 | Model 3 | Allocated Points (N - rank) |
|------|---------|---------|---------|-----------------------------|
| 1    | A       | D       | E       | 5 - 1 = 4                   |
| 2    | B       | C       | A       | 5 - 2 = 3                   |
| 3    | C       | A       | B       | 5 - 3 = 2                   |
| 4    | D       | B       | D       | 5 - 4 = 1                   |
| 5    | E       | E       | C       | 5 - 5 = 0                   |

Based on the allocated points for each of the items, we sum the points up to get our Borda Count.

| Item | Model 1 | Model 2 | Model 3 | Borda Count  |
|------|---------|---------|---------|--------------|
| A    | 4       | 2       | 3       | 9            |
| B    | 3       | 1       | 2       | 6            |
| C    | 2       | 3       | 0       | 5            |
| D    | 1       | 4       | 1       | 6            |
| E    | 0       | 0       | 4       | 4            |

New ranking: A > B, D > C > E


Lets implement this method below.

In [11]:
# Let's create a new dataframe to calculate ranking and borda count
rank_df = pd.DataFrame({
    "ItemID": item_idx2id,
})

total_items = len(rank_df) # 1651 items

# Obtain points (inverse of rank) of the items based on the BPR score
rank_df["BPR Score"] = bpr_scores
rank_df["BPR Rank"] = rank_df["BPR Score"].rank(ascending=False).astype(int) # Get Rank where 1 = Top recommendation
rank_df["BPR Points"] = total_items - rank_df["BPR Rank"] # Get points by calculating ('Total Item count' - 'Rank')

# Do likewise for WMF
rank_df["WMF Score"] = wmf_scores
rank_df["WMF Rank"] = rank_df["WMF Score"].rank(ascending=False).astype(int) # Get Rank where 1 = Top recommendation
rank_df["WMF Points"] = total_items - rank_df["WMF Rank"] # Get points by calculating ('Total Item count' - 'Rank')

# Get Borda Count by summing up points of BPR and WMF
rank_df["Borda Count"] = rank_df["BPR Points"] + rank_df["WMF Points"]

# Round decimal places for readability purposes
rank_df = rank_df.round(3)

# Now let's take a look at the table with Borda Count 
display(rank_df.head(5))

,ItemID,BPR Score,BPR Rank,BPR Points,WMF Score,WMF Rank,WMF Points,Borda Count
0,381,1.302,450,1201,3.798,204,1447,2648
1,602,-0.050,762,889,1.794,786,865,1754
2,431,-1.202,1069,582,3.317,322,1329,1911
3,875,1.631,388,1263,1.532,874,777,2040
4,182,1.536,405,1246,3.221,352,1299,2545


Now that we have Borda Count, let's rerank this list and to provide the ensembled model's recommendation.

In [13]:
# Introduce reranked dataframe for borda count
reranked_df = rank_df.sort_values("Borda Count", ascending=False)

# Let's take a look at the ensembled top 5 recommendations and their respective ranks
display("Re-ranked Top K Item recommendations", reranked_df.head(5))

'Re-ranked Top K Item recommendations'

,ItemID,BPR Score,BPR Rank,BPR Points,WMF Score,WMF Rank,WMF Points,Borda Count
279,402,5.336,1,1650,5.169,15,1636,3286
305,181,4.564,13,1638,5.360,8,1643,3281
425,237,5.266,3,1648,5.064,25,1626,3274
262,275,4.832,7,1644,5.076,22,1629,3273
152,313,4.077,35,1616,6.046,1,1650,3266


The top recommendation ItemID **181**, was ranked **7** on BPR and **12** on WMF.

By doing Borda Count, we are able to aggregate model recommendations.

Next, let's add the recommendations into the genre distribution dataframe to compare its results to the base models.

In [14]:
borda_count_topk = reranked_df["ItemID"].values[:TOPK] # Get top K (50) Item IDs

borda_df = item_df.loc[[int(i) for i in borda_count_topk]] # Filter genre data frame by the top item IDs

# Add Borda Count results into 'combined_df' dataframe for comparison
combined_df["Borda Count Sum"] = borda_df.select_dtypes(np.number).sum() # group by genre, and calculate sum of each genre
combined_df["Borda Count %"] = combined_df["Borda Count Sum"] / combined_df["Borda Count Sum"].sum() * 100 # Calculate percentage of sum to total
combined_df["Borda Count %"] = combined_df["Borda Count %"].round(1) # rounding for readability purposes

# Let's take a look at the genre distribution of train data, BPR, WMF and the newly added Borda Count
display("Borda Count Recommendations Distribution", combined_df[["Train Data %", "BPR %", "WMF %", "Borda Count %"]])

'Borda Count Recommendations Distribution'

,Train Data %,BPR %,WMF %,Borda Count %
Drama,24.9,12.4,31.0,18.2
Comedy,13.7,25.7,5.2,16.5
Romance,11.2,25.7,14.7,18.2
Action,10.5,10.6,8.6,8.3
Thriller,10.2,5.3,7.8,8.3
Adventure,6.7,6.2,4.3,5.8
War,5.3,4.4,8.6,4.1
Crime,4.2,0.9,3.4,0.8
Sci-Fi,3.2,5.3,4.3,5.0
Mystery,2.8,0.0,1.7,0.8


In the next section, we will further add more models to the ensemble.

## 3. Adding more models to the Borda Count ensemble

We can easily add more models to the ensemble by training them and adding them. One approach is to train a model with different initializations using different random seeds. By adding multiple similar models of different random seeds (`seed=123`), some models could perform better for a set of users, while other models could perform better for another set of users.

Another way is to change the number of latent factors `k`. By achieving better performance on a subset of data for each of the models, we are able to improve the performance of the ensembled model as a whole.

By ensembling these models, we could potentially achieve better performance when combined.

Let's try adding a few more similar models with different random seed initializations.

In [15]:
# WMF models with different seeds
wmf_model_123 = WMF(name="WMF_123", k=10, max_iter=300, a=1.0, b=0.1, learning_rate=0.001, lambda_u=0.01, lambda_v=0.01, seed=123)
wmf_model_456 = WMF(name="WMF_456", k=20, max_iter=300, a=1.0, b=0.1, learning_rate=0.001, lambda_u=0.01, lambda_v=0.01, seed=456)
wmf_model_789 = WMF(name="WMF_789", k=30, max_iter=300, a=1.0, b=0.1, learning_rate=0.001, lambda_u=0.01, lambda_v=0.01, seed=789)
wmf_model_888 = WMF(name="WMF_888", k=40, max_iter=300, a=1.0, b=0.1, learning_rate=0.001, lambda_u=0.01, lambda_v=0.01, seed=888)
wmf_model_999 = WMF(name="WMF_999", k=50, max_iter=300, a=1.0, b=0.1, learning_rate=0.001, lambda_u=0.01, lambda_v=0.01, seed=999)

models = [wmf_model_123, wmf_model_456, wmf_model_789, wmf_model_888, wmf_model_999]

# Let's run an experiment to take a look at how different these models are, with just different random seeds!
experiment = Experiment(rs, models, metrics, user_based=True).run()

I0000 00:00:1728143155.204175   49677 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355



[WMF_123] Training started!


100%|██████████| 300/300 [00:05<00:00, 59.84it/s, loss=173]


Learning completed!

[WMF_123] Evaluation started!


Ranking: 100%|██████████| 940/940 [00:00<00:00, 3221.07it/s]
I0000 00:00:1728143160.582240   49677 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


...


[WMF_999] Training started!


100%|██████████| 300/300 [00:05<00:00, 51.53it/s, loss=102] 


Learning completed!

[WMF_999] Evaluation started!


Ranking: 100%|██████████| 940/940 [00:00<00:00, 2474.52it/s]


TEST:
...
        | Precision@50 | Recall@50 | Train (s) | Test (s)
------- + ------------ + --------- + --------- + --------
WMF_123 |       0.1750 |    0.5103 |    5.0685 |   0.2940
WMF_456 |       0.1779 |    0.5220 |    5.3761 |   0.5020
WMF_789 |       0.1734 |    0.5129 |    5.1717 |   0.4665
WMF_888 |       0.1706 |    0.5031 |    5.7383 |   0.4481
WMF_999 |       0.1656 |    0.4944 |    5.8949 |   0.3819



Based on the results, we can see that even within the same model, the results can vary. 

Let's try ensembling all these models together into 1 single model by Borda Count, and look at its recommendations.

In [16]:
# Let's create a different dataframe to calculate ranking and borda count
rank_2_df = pd.DataFrame({
    "ItemID": item_idx2id,
})

# Add a column named 'Ensembled WMF Model'
rank_2_df["Ensembled WMF Model"] = 0

# Calculate the points (inverse of rank) for each of the models and accumulate them into the 'Enhanced Borda Count' column
for model in models:
    name = model.name
    recommendations, scores = model.rank(UIDX)
    rank_2_df[name + "_rating"] = scores
    rank_2_df[name + "_rank"] = rank_2_df[name + "_rating"].rank(ascending=False).astype(int)
    rank_2_df[name + "_points"] = total_items - rank_2_df[name + "_rank"]
    rank_2_df["Ensembled WMF Model"] = rank_2_df["Ensembled WMF Model"] + rank_2_df[name + "_points"]

# Round results for readability
rank_2_df = rank_2_df.round(3)

# Let's sort and view the top recommendations!
print("Model score calculation:")
display(rank_2_df[["Ensembled WMF Model"]].sort_values("Ensembled WMF Model", ascending=False).head(10))

Model score calculation:


,Ensembled WMF Model
37,8229
197,8178
152,8151
279,8126
532,8122
156,8121
147,8105
522,8085
61,8076
670,8063


In [17]:
# Now, let's add them to the combined dataframe for comparison with earlier models
enhanced_borda_count_topk = list(rank_2_df.sort_values("Ensembled WMF Model", ascending=False)["ItemID"].values[:TOPK])
enhanced_borda_df = item_df.loc[[int(i) for i in enhanced_borda_count_topk]]

combined_df["Ensembled WMF Model Sum"] = enhanced_borda_df.select_dtypes(np.number).sum()
combined_df["Ensembled WMF Model %"] = combined_df["Ensembled WMF Model Sum"] / combined_df["Ensembled WMF Model Sum"].sum() * 100
combined_df["Ensembled WMF Model %"] = combined_df["Ensembled WMF Model %"].round(1)

# Let's compare the recommendation distribution
display("Combined Recommendations Distribution", combined_df[["Train Data %", "BPR %", "WMF %", "Borda Count %", "Ensembled WMF Model %"]])

'Combined Recommendations Distribution'

,Train Data %,BPR %,WMF %,Borda Count %,Ensembled WMF Model %
Drama,24.9,12.4,31.0,18.2,23.9
Comedy,13.7,25.7,5.2,16.5,13.7
Romance,11.2,25.7,14.7,18.2,15.4
Action,10.5,10.6,8.6,8.3,10.3
Thriller,10.2,5.3,7.8,8.3,6.8
Adventure,6.7,6.2,4.3,5.8,7.7
War,5.3,4.4,8.6,4.1,4.3
Crime,4.2,0.9,3.4,0.8,5.1
Sci-Fi,3.2,5.3,4.3,5.0,5.1
Mystery,2.8,0.0,1.7,0.8,3.4


Now that we have touched on simple borda count, let's see how we could use other methods and popular packages such as **scikit-learn** to do advanced model ensembling.

## 4. Model Ensembling via Regression Models

We could continue by thinking of this as a meta-learning problem. We could treat recommendations of each base model as features and train a meta-learner to predict the final recommendation.

This could be any ML model such as a Linear Regression, Random Forest, Gradient Boosting, or even a Neural Network.

In this example, we will use a simple Linear Regression model to predict the final recommendation.

We will teach a model to learn from the different outputs of the base models of WMF.

##### 4.1 Prepare Data

In [18]:
# First, lets create training and test data dataframes
training_df = pd.DataFrame(zip(*train_set.uir_tuple)) # Add 'User Index', 'Item Index', 'Rating' triples as records in dataframe
training_df.columns = ['user_idx', 'item_idx', 'ground_score'] # Set column names

# Get all possible user_index, item_index combinations, add them into dataframe for inference
all_df = pd.DataFrame({
    "user_idx": [user_idx for user_idx in range(train_set.num_users) for _ in range(train_set.num_items)],
    "item_idx": [item_idx for _ in range(train_set.num_users) for item_idx in range(train_set.num_items)],
})
all_df['item_id'] = all_df.apply(lambda row: item_idx2id[int(row['item_idx'])], axis=1) # Add 'Item ID' column into dataframe by converting 'Item Index' to 'Item ID'

# Lets get all the scores for the models trained in Part 3.

# For each model, we add individual predicted ratings by individual models to training and test dataframes
for model in models:
    name = model.name
    training_df[name + "_score"] = training_df.apply(lambda row: model.score(int(row['user_idx']), int(row['item_idx'])), axis=1) # for training
    all_df[name + "_score"] = all_df.apply(lambda row: model.score(int(row['user_idx']), int(row['item_idx'])), axis=1) # for inference

# Let's pick out the 5 features - predicted ratings from the 5 models trained
X_train = training_df[['WMF_123_score', 'WMF_456_score', 'WMF_789_score', 'WMF_888_score', 'WMF_999_score']]
y_train = training_df['ground_score'] # use ground truth to train this linear regression model
X_inference = all_df[['WMF_123_score', 'WMF_456_score', 'WMF_789_score', 'WMF_888_score', 'WMF_999_score']] # all data, used to predict values for ranking

display("Training features", X_train.head(3)) # predicting ratings as features
display("Target values", y_train.head(3)) # ground truth ratings
display("Inference Data", X_inference.head(3)) # all inference data

'Training features'

,WMF_123_score,WMF_456_score,WMF_789_score,WMF_888_score,WMF_999_score
0,2.110018,3.753257,3.692900,3.763934,3.974546
1,2.791631,2.339661,2.418044,2.738071,3.102871
2,3.750932,3.537887,3.441126,3.396540,3.855725


'Target values'

0    4.0
1    3.0
2    4.0
Name: ground_score, dtype: float64

'Inference Data'

,WMF_123_score,WMF_456_score,WMF_789_score,WMF_888_score,WMF_999_score
0,2.110018,3.753257,3.692900,3.763934,3.974546
1,0.807434,-0.628530,-0.064267,-0.289080,-0.587279
2,1.648493,1.840384,1.829896,1.870474,1.534385


Now that we have already prepared the data for fitting into a **scikit-learn** model, let's first try to train a Linear Regression model

##### 4.2 Fitting Linear Regression Model

In [19]:
# Let's now fit into a Linear Regression model
regr = linear_model.LinearRegression(fit_intercept=False) # force model to only use predictions from WMF models
regr.fit(X_train, y_train) # train the model

# Input: 5 base model predicted ratings. Output: final predicted rating based on linear regression
y_pred = regr.predict(X_inference) # Get predictions based on trained model

all_df["Linear Regression Prediction"] = y_pred # create a column in `test_df` for the predictions

# Get Top K ratings from predictions
all_df = all_df.sort_values("Linear Regression Prediction", ascending=False) # sort by predicted ratings
top_item_ids = all_df[all_df['user_idx'] == UIDX]['item_id'].values[:TOPK] # filter top K (50 as set in Section 2.3)

# Place them into the comparison distribution dataframe
linear_regression_df = item_df.loc[[int(i) for i in top_item_ids]] # Get genres of ratings
combined_df["Linear Regression Sum"] = linear_regression_df.select_dtypes(np.number).sum() # group by genre and sum them up
combined_df["Linear Regression %"] = combined_df["Linear Regression Sum"] / combined_df["Linear Regression Sum"].sum() * 100 # get percentages of (genre sum / whole sum)

combined_df["Linear Regression %"] = combined_df["Linear Regression %"].round(1) # round values for readability

print(regr.coef_)
print(regr.intercept_)

# Now let's take a look at how the genre distribution is
display("Combined Recommendations Distribution", combined_df[["Train Data %", "BPR %", "WMF %", "Borda Count %", "Ensembled WMF Model %", "Linear Regression %"]])

[-0.18445674 -0.13890123  0.1438685   0.43231982  0.86287963]
0.0


'Combined Recommendations Distribution'

,Train Data %,BPR %,WMF %,Borda Count %,Ensembled WMF Model %,Linear Regression %
Drama,24.9,12.4,31.0,18.2,23.9,27.6
Comedy,13.7,25.7,5.2,16.5,13.7,8.6
Romance,11.2,25.7,14.7,18.2,15.4,16.2
Action,10.5,10.6,8.6,8.3,10.3,6.7
Thriller,10.2,5.3,7.8,8.3,6.8,8.6
Adventure,6.7,6.2,4.3,5.8,7.7,7.6
War,5.3,4.4,8.6,4.1,4.3,4.8
Crime,4.2,0.9,3.4,0.8,5.1,4.8
Sci-Fi,3.2,5.3,4.3,5.0,5.1,4.8
Mystery,2.8,0.0,1.7,0.8,3.4,1.9


Explanation, interpretation. Last model contributes the most >>

We have successfully fit into a Linear Regression model using the 5 WMF base models built on Cornac.

You could continue using different regression models on **scikit-learn** to further predict ratings.

In the section below, we will train a Random Forest model.

##### 4.3 Fitting Random Forest Model

We reuse the same training data. >> Technically can use any machine learning model.


In [20]:
# Let's now train a Random Forest model
randomforest_model = RandomForestRegressor(n_estimators=50, random_state=42) 
randomforest_model.fit(X_train, y_train) # Train the model

# Input: 5 base model predicted ratings. Output: final predicted rating based on random forest
y_pred = randomforest_model.predict(X_inference)

all_df["Random Forest Score"] = y_pred # create a column in `all_df` for the predictions

# Get Top K ratings from predictions
all_df = all_df.sort_values("Random Forest Score", ascending=False) # sort by predicted ratings
top_item_ids = all_df[all_df['user_idx'] == UIDX]['item_id'].values[:TOPK] # filter top K (50 as set in Section 2.3)

# Place them into the comparison distribution dataframe
random_forest_df = item_df.loc[[int(i) for i in top_item_ids]] # Get genres of ratings
combined_df["Random Forest Sum"] = random_forest_df.select_dtypes(np.number).sum() # group by genre and sum them up
combined_df["Random Forest %"] = combined_df["Random Forest Sum"] / combined_df["Random Forest Sum"].sum() * 100 # get percentages of (genre sum / whole sum)

combined_df["Random Forest %"] = combined_df["Random Forest %"].round(1) # round values for readability

# Now let's take a look at how the genre distribution is
display("Combined Recommendations Distribution", combined_df[["Train Data %", "BPR %", "WMF %", "Borda Count %", "Ensembled WMF Model %", "Linear Regression %", "Random Forest %"]])

'Combined Recommendations Distribution'

,Train Data %,BPR %,WMF %,Borda Count %,Ensembled WMF Model %,Linear Regression %,Random Forest %
Drama,24.9,12.4,31.0,18.2,23.9,27.6,16.3
Comedy,13.7,25.7,5.2,16.5,13.7,8.6,18.4
Romance,11.2,25.7,14.7,18.2,15.4,16.2,8.2
Action,10.5,10.6,8.6,8.3,10.3,6.7,11.2
Thriller,10.2,5.3,7.8,8.3,6.8,8.6,10.2
Adventure,6.7,6.2,4.3,5.8,7.7,7.6,4.1
War,5.3,4.4,8.6,4.1,4.3,4.8,1.0
Crime,4.2,0.9,3.4,0.8,5.1,4.8,5.1
Sci-Fi,3.2,5.3,4.3,5.0,5.1,4.8,5.1
Mystery,2.8,0.0,1.7,0.8,3.4,1.9,4.1


What happened in Section 4 was that we trained a linear regression model and random forest model to learn from the different base models, adapting to the changes in features based on how these base models are configured to be.

---

Ensemble learning doesn't stop here. You could continue on by: 
- Trying different Cornac base models
- Trying different ensemble methods
- Tweaking around base models and different parameters within them

By having different base models with different specializations, model ensembling could leverage on the strengths of different models. Effectiveness of these models could depend on many factors such as diversity and quality of base models, dataset size and quality.

It is also important to note that while model ensembling theoretically provides superior performance, there could be instances where base models outperforms ensembled models. Model ensembling also requires more computational resources as well. Therefore, we should consider striking a balance between performanace and computational costs.

---

So what constitutes to a good ensembled model? Which base models and configurations are ideal? These are topics that require further experimentation and discussion.

## 5. Experimental Comparison

In [21]:
# N = train_set.num_users

from cornac.metrics import RankingMetric

# def compute_recall(retrieved_neighbors, true_neighbors):
#     total = 0
#     for retrieved, true in zip(retrieved_neighbors, true_neighbors):
#         total += len(set(retrieved) & set(true))
#     return total / (N * TOPK)

def compute_rank_metric(rank_metric, gt_pos, pd_rank):
    if isinstance(rank_metric, RankingMetric) == False:
        raise ValueError(f"{rank_metric.name} is not a valid Ranking Metric. Please use a valid ranking metric.")
    return rank_metric.compute(gt_pos, pd_rank)

In [34]:
metrics = [Precision(k=50), Recall(k=50)] # Set metrics for experiment

# Group by user id, calculate recall for each user

retrieved_neighbors = []

for uidx in range(N):
    top_item_ids = all_df[all_df['user_idx'] == uidx]['item_id'].values[:TOPK] # filter top K (50 as set in Section 2.3)
    retrieved_neighbors.append(top_item_ids)
    



['a', 'b']


Let's add the base models (BPR, WMF), Borda count, Enhanced WMF model into `all_df` so that we can compare them with the ranking metrics set.

In [41]:
additional_models = [bpr_model, wmf_model]

for model in additional_models:
    name = model.name
    all_df[name + "_score"] = all_df.apply(lambda row: model.score(int(row['user_idx']), int(row['item_idx'])), axis=1) # for inference

# TODO: Add borda acount, enhanced WMF. 
# Will need to calculate scores of all users, group users then calculate for borda count. Likewise for enhanced WMF.

# Let's create a new dataframe to calculate ranking and borda count
# rank_df = pd.DataFrame({
#     "ItemID": item_idx2id,
# })

# total_items = len(rank_df) # 1651 items

# # Obtain points (inverse of rank) of the items based on the BPR score
# rank_df["bpr_rank"] = rank_df["bpr_score"].rank(ascending=False).astype(int) # Get Rank where 1 = Top recommendation
# rank_df["bpr_points"] = total_items - rank_df["bpr_rank"] # Get points by calculating ('Total Item count' - 'Rank')

# # Do likewise for WMF
# rank_df["WMF Score"] = wmf_scores
# rank_df["WMF Rank"] = rank_df["WMF Score"].rank(ascending=False).astype(int) # Get Rank where 1 = Top recommendation
# rank_df["WMF Points"] = total_items - rank_df["WMF Rank"] # Get points by calculating ('Total Item count' - 'Rank')

# # Get Borda Count by summing up points of BPR and WMF
# rank_df["Borda Count"] = rank_df["BPR Points"] + rank_df["WMF Points"]

In [42]:
display(all_df)

,user_idx,item_idx,item_id,WMF_123_score,WMF_456_score,WMF_789_score,WMF_888_score,WMF_999_score,Linear Regression Prediction,Random Forest Score,BPR_score,WMF_score
1340448,811,1487,104,-0.061779,-0.130752,0.004339,-0.020222,-0.000564,0.020952,5.0,-1.736096,-0.041820
547229,331,748,85,0.909898,1.678483,0.672016,1.233451,1.073962,1.155647,5.0,0.858288,1.677042
1276851,773,628,200,0.612809,-0.014905,0.321336,0.201020,0.719644,0.643135,5.0,0.632851,0.158951
1537064,930,1634,1604,0.015906,-0.000650,0.047789,-0.043355,0.035993,0.016346,5.0,-1.964396,0.022136
1074400,650,1250,368,0.274818,0.271891,0.415881,0.971956,0.556839,0.872056,5.0,-1.060161,0.393766
...,...,...,...,...,...,...,...,...,...,...,...,...
878977,532,645,322,1.410058,0.168109,-1.481861,-0.553036,-3.330819,-3.609823,1.0,2.048553,0.559598
1026121,621,850,126,-0.402692,-1.280105,-2.845929,-1.798385,-3.106802,-3.615626,1.0,2.720750,-1.469236
313897,190,207,241,0.536907,0.049187,-1.401366,-1.665010,-3.001801,-3.617491,1.0,1.159508,0.339348
1160674,703,21,303,-0.099273,-0.552296,0.059405,-1.604818,-3.512952,-3.621476,1.0,1.010203,0.667266


Now, let's calculate the metrics and commpare them!

In [43]:
from cornac.metrics import Recall

test_data_df = pd.DataFrame(zip(*test_set.uir_tuple)) # adding test training data into dataframe
test_data_df.columns = ['user_idx', 'item_idx', 'rating'] # adding column names to the data

test_data_useridx_grouped = test_data_df.groupby('user_idx').item_idx.apply(list) # group by user index to list of item indexes feedbacked for each user

metrics = [Precision(k=50), Recall(k=50)] # Set metrics for experiment
model_columns = ["BPR_score", "WMF_score", "WMF_123_score", "WMF_456_score", "WMF_789_score", "WMF_888_score", "WMF_999_score", "Linear Regression Prediction", "Random Forest Score"]

result_cols = ["Model"] + [metric.name for metric in metrics]
model_results = []

for model_col in model_columns:
    all_df = all_df.sort_values(model_col, ascending=False) # sort by predicted ratings
    predicted_ids = [all_df[all_df['user_idx'] == uidx]['item_idx'].values[:TOPK].astype(int) for uidx in range(train_set.num_users)]

    metric_results = [model_col]

    for metric in metrics:
        total = 0
        count = 0

        for uidx in range(train_set.num_users):
            if uidx not in test_data_useridx_grouped:
                continue
            sample_gt_pos = test_data_useridx_grouped[uidx] # ground truth data
            sample_pd_rank = predicted_ids[uidx].tolist() # predicted ranking data
            
            val = compute_rank_metric(metric, sample_gt_pos, sample_pd_rank) # compute recall
            total += val
            count += 1

        total = total / count # average
        metric_results.append(total)
    
    model_results.append(metric_results)

display(pd.DataFrame(model_results, columns=result_cols))

,Model,Precision@50,Recall@50
0,BPR_score,0.099298,0.364242
1,WMF_score,0.103511,0.375467
2,WMF_123_score,0.103255,0.372346
3,WMF_456_score,0.100979,0.376920
4,WMF_789_score,0.092830,0.353123
5,WMF_888_score,0.087213,0.333365
6,WMF_999_score,0.081021,0.320774
7,Linear Regression Prediction,0.077766,0.315686
8,Random Forest Score,0.019617,0.053716


We have compared models in part 3. Let's 